# 15.095. Homework 4

Kim-Anh-Nhi Nguyen

# Question 3

## (a) Data preprocessing

In [40]:
# imports
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
#from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error

%matplotlib inline


In [2]:
# load data
items = pd.read_csv("items.csv")
sales = pd.read_csv("sales.csv")
info = pd.read_csv("sideinformation.csv")

First, we look at what the 3 tables look like.

In [3]:
sales.head()

,date,item_nbr,ondisplay,onpromotion,unit_sales
0,2017-01-02,103665,False,False,0.0
1,2017-01-02,105574,False,False,0.0
2,2017-01-02,105575,True,False,3.0
3,2017-01-02,105577,True,False,1.0
4,2017-01-02,105737,True,False,1.0


In [4]:
# check there are 100 unique items
print(sales['item_nbr'].unique().size)

100


In [5]:
items.head()

,item_nbr,family,class,perishable,cost,price
0,103665,BREAD/BAKERY,2712,1,0.72,4.00
1,105574,GROCERY I,1045,0,1.97,3.33
2,105575,GROCERY I,1045,0,1.02,3.72
3,105577,GROCERY I,1045,0,2.21,4.89
4,105737,GROCERY I,1044,0,2.35,3.71


In [6]:
# check how many unique items, families and classes
print(items['item_nbr'].unique().size)
print(items['family'].unique().size)
print(items['class'].unique().size)

100
12
47


In [7]:
info.head()

,date,isHoliday,oilPrice
0,2017-01-02,True,52.36
1,2017-01-03,False,53.26
2,2017-01-04,False,53.06
3,2017-01-05,False,53.62
4,2017-01-06,False,52.41


### We encode the categorical variables as integers

In [8]:
# label encoder for item_nbr, family and class

le1 = preprocessing.LabelEncoder()
le1.fit(list(items['item_nbr'].unique()))
items['item_nbr_encoded'] = le1.transform(items['item_nbr'])

le2 = preprocessing.LabelEncoder()
le2.fit(list(items['family'].unique()))
items['family_encoded'] = le2.transform(items['family'])

le3 = preprocessing.LabelEncoder()
le3.fit(list(items['class'].unique()))
items['class_encoded'] = le3.transform(items['class'])

### Now, we can merge the 3 tables

In [9]:
#merge items and sales, sales on the left
df = pd.merge(sales, items, on = 'item_nbr', how = 'left')

In [10]:
# merge with info table
df = pd.merge(df, info, on = 'date', how = 'left')

### More Data Processing: 
1. We create 3 columns to take the date into account
2. We round the sales as integers, because there is numerical float issues for some rows (e.g. 3.999999998 instead of 4.0) 

In [11]:
#column year, month, day
df['year'] = df['date'].str[0:4]
df['month'] = df['date'].str[5:7]
df['day'] = df['date'].str[8:10]

In [12]:
df['unit_sales'] = round(df['unit_sales'], 0)

### We export the final merged table

In [13]:
df.to_csv("final_data_with_date.csv")

### We separate the data into a train and a test set
- The training set contains all the observations from 2017-01-02 to 2017-08-14
- The testing set contains all the observations from our target date only, i.e., 2017-08-15
- the target variable for each set (the y variable) is the `sales` column
- Finally, we export the different tables, so we can use them for predictions

In [14]:
df_train, df_test = df.loc[df['date'] != '2017-08-15'], df.loc[df['date'] == '2017-08-15']
print(len(df_train), len(df_test))

22500 100


In [15]:
df2_train = df_train.drop(['item_nbr', 'family', 'class', 'date', 'unit_sales'], axis = 1)
df2_test = df_test.drop([ 'item_nbr', 'family', 'class', 'date', 'unit_sales'], axis = 1)

In [16]:
y_train, y_test = df_train[['unit_sales']], df_test[['unit_sales']]

In [17]:
df2_train.to_csv("X_train.csv")
df2_test.to_csv("X_test.csv")
y_train.to_csv("y_train.csv")
y_test.to_csv("y_test.csv")

---
## (b) Model training: Decision tree

In [18]:
# load data
df = pd.read_csv("final_data_with_date.csv")
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")
y_test = pd.read_csv("y_test.csv")

# we drop the 'Unnamed: 0' column (old rows in the original tables)
X_train = X_train.drop(['Unnamed: 0'], axis = 1)
X_test = X_test.drop(['Unnamed: 0'], axis = 1)
y_train = y_train.drop(['Unnamed: 0'], axis = 1)
y_test = y_test.drop(['Unnamed: 0'], axis = 1)

In [19]:
from sklearn.externals import joblib

def savemodel(filename, model):
    # Save to file in the current working directory
    path = filename + ".joblib"
    joblib_file = path
    joblib.dump(model, joblib_file)

def loadmodel(joblib_file):
    # Load from file
    joblib_model = joblib.load(joblib_file)
    return(joblib_model)

In [20]:
# parameters that we are going to tune
DT_param_grid = { 'max_depth':[2, 3, 4, 5, 10, 20, 50],
                'min_samples_split': [2, 10, 50, 100],
                 'min_samples_leaf': [1, 5, 10]
                }

In [21]:
DT = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid = DT_param_grid, 
                              scoring='neg_mean_absolute_error', 
                              cv=10, 
                              verbose=10, n_jobs = 16)
DT.fit(X_train, y_train)
DTbest = DT.best_estimator_
print(DTbest)

Fitting 10 folds for each of 84 candidates, totalling 840 fits


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    8.7s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    9.2s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    9.9s
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:   10.2s
[Parallel(n_jobs=16)]: Done  53 tasks      | elapsed:   10.6s
[Parallel(n_jobs=16)]: Done  66 tasks      | elapsed:   11.1s
[Parallel(n_jobs=16)]: Done  81 tasks      | elapsed:   11.6s
[Parallel(n_jobs=16)]: Done  96 tasks      | elapsed:   12.1s
[Parallel(n_jobs=16)]: Done 113 tasks      | elapsed:   12.7s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:   13.3s
[Parallel(n_jobs=16)]: Done 149 tasks      | elapsed:   13.9s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:   14.6s
[Parallel(n_jobs=16)]: Done 189 tasks      | elapsed:   15.2s
[Parallel(n_jobs=16)]: Done 210 tasks      | elapsed:   16.1s
[Parallel(n_jobs=16)]: Done 233 tasks      | elapsed:  

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=10,
           min_samples_split=100, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')


[Parallel(n_jobs=16)]: Done 840 out of 840 | elapsed:   46.7s finished


In [22]:
# save the model
savemodel("DT", DTbest)

## (c) Model performance, let's compare with a baseline model

In [41]:
# make predictions
ypred = pd.DataFrame(DTbest.predict(X_test))

# but sales volumes are integers so, we round each prediction to the closest integer
ypred_round = round(ypred)

# get the MAE
MAE = mean_absolute_error(y_test, ypred_round)

print('MAE with rounded predictions:', MAE)


MAE with rounded predictions: 1.49


In [24]:
# baseline model predicts that sales for August 15 are exactly the sales from the previous day 
y_baseline = y_train.iloc[-100:, :]

In [25]:
# get the MAE
MAE_b = mean_absolute_error(y_test, y_baseline)
print("MAE of baseline: ", MAE_b)

MAE of baseline:  1.94


# Export data for the optimization problem
We will switch to Julia in order to solve the optimization problem.

To do so, we will need the values `y_train`, `y_pred`, `y_baseline`, as well as the leaves in the decision tree we built that correspond to each value y.

In [26]:
# predictions y hat:
ypred_round.to_csv("y_pred.csv")
y_baseline.to_csv("y_baseline.csv")

In [38]:
# transform boolean into integers
X_test["ondisplay"] = X_test["ondisplay"].astype('int64')
X_test["onpromotion"] = X_test["onpromotion"].astype('int64')
X_test["isHoliday"] = X_test["isHoliday"].astype('int64')
X_test.to_csv("X_test_int.csv")

In [27]:
# export the baseline data for the optimization model in Julia
X_baseline = X_train.iloc[-100:, :]
# transform boolean into integers
X_baseline["ondisplay"] = X_baseline["ondisplay"].astype('int64')
X_baseline["onpromotion"] = X_baseline["onpromotion"].astype('int64')
X_baseline["isHoliday"] = X_baseline["isHoliday"].astype('int64')
X_baseline.to_csv("X_baseline.csv")

C:\Users\utilisateur\.julia\v0.6\Conda\deps\usr\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\utilisateur\.julia\v0.6\Conda\deps\usr\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\utilisateur\.julia\v0.6\Conda\deps\usr\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [28]:
# get the leaf' number where each observation belongs
leaf_belong_train = DTbest.apply(X_train)
leaf_belong_test = DTbest.apply(X_test)

In [29]:
# pull out the leaves for both data sets: each row is a pair of {unit_sales, leaf where the corresponding observation was}

train_leaves = []
for i in range(len(y_train)):
     train_leaves.append((list(y_train['unit_sales'])[i], leaf_belong_train[i]))
        
test_leaves = []
for i in range(len(y_test)):
     test_leaves.append((list(y_test['unit_sales'])[i], leaf_belong_test[i]))

In [30]:
# export the leaves 
train_leaves = np.array(train_leaves)
np.savetxt("train_leaves.csv", train_leaves, delimiter=",")

test_leaves = np.array(test_leaves)
np.savetxt("test_leaves.csv", test_leaves, delimiter=",")

In [32]:
# function to get all unit_sales values belonging to a leaf

def fetch_points_in_leaf(leaf, train_leaves):
    n = len(train_leaves)
    in_leaf = []
    for x in train_leaves:
        if x[1] == leaf:
            in_leaf.append(x[0])
    return(in_leaf)

In [33]:
# for the purpose to get a matrix where every row has the same length
# we add -1 to complete the rows
def complete_list(l, length):
    if len(l) < length:
        new_list = [-1 for i in range(length)]
        for i in range(len(l)):
            new_list[i] = l[i]
    return(new_list)

In [34]:
# gives the unit_sales from the training set which are in a particular leaf
def get_leaf_points(train_leaves, test_leaves):
    test_leaf_data = []
    for x in test_leaves:
        leaf = x[1] # leaf where y_pred[i] belongs
        in_leaf = fetch_points_in_leaf(leaf, train_leaves)
        test_leaf_data.append( complete_list(in_leaf, len(train_leaves)) )     
    return(test_leaf_data)

In [35]:
# get the weights corresponding to a leaf
# equals to 1/(number of points in the leaf)

def get_weight(train_leaves, test_leaves):
    weights = []
    leaf_points = get_leaf_points(train_leaves, test_leaves)
    for x in leaf_points:
        count = 0
        for i in range(len(x)):
            if x[i] != -1:
                count +=1
        weights.append(1/count)
    return(weights)

In [36]:
test_leaf_data = get_leaf_points(train_leaves, test_leaves)
weights = get_weight(train_leaves, test_leaves)

In [37]:
# export the leaves data

test_leaf_data = np.array(test_leaf_data)
test_leaf_data = np.transpose(test_leaf_data) # transpose because otherwise, the csv will have 22500 columns VS 100 columns
np.savetxt("test_leaf_data_T.csv", test_leaf_data, delimiter=",")
weights = np.array(weights)
np.savetxt("weights.csv", weights, delimiter=",")